In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 在GCP上进行端到端ML：MLOps阶段6：使用Vertex AI原始预测开始使用TensorFlow serving功能

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_raw_predict.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_raw_predict.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage6/get_started_with_raw_predict.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>          
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用`Vertex AI原始预测`将原始的HTTP内容直接发送给部署在`Vertex AI终端节点`上的模型。

例如，预构建的`Vertex AI`部署容器的HTTP服务器不支持用于TensorFlow 1.x估计器的HTTP请求主体。使用原始预测，可以通过HTTP服务器发送原始内容，该内容被呈现给模型输入，不经过规范处理。

### 目的

在本教程中，您将学习如何在 `Vertex AI Endpoint` 资源上使用 `Vertex AI Raw Prediction`。

本教程使用以下 Google Cloud ML 服务和资源：

- `Vertex AI Raw Prediction`
- `Vertex AI Models`
- `Vertex AI Endpoints`

执行的步骤包括：

- 下载一个预训练的用于 TensorFlow 1.x 估计器的表格分类模型文件。
- 将 TensorFlow 估计器模型上传为 `Vertex AI Model` 资源。
- 创建一个 `Endpoint` 资源。
- 部署 `Model` 资源到 `Endpoint` 资源。
- 对部署到 `Endpoint` 资源的 `Model` 资源实例进行在线原始预测。

### 数据集

本教程使用了一个从公共云存储桶中预先训练好的表格分类模型，该模型是在企鹅数据集（https://cloud.google.com/bigquery/public-data）上进行训练的。该数据集的版本用于预测物种。

成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage
定价](https://cloud.google.com/storage/pricing)，并使用 [定价
计算器](https://cloud.google.com/products/calculator/)
根据您的预期使用量生成成本估算。

## 安装

安装以下包以执行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q
! pip3 install --upgrade google-cloud-pipeline-components $USER_FLAG -q
! pip3 install tensorflow-hub $USER_FLAG -q

重启内核

安装了额外的包后，您需要重新启动笔记本内核，以便它能够找到这些包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU运行时

*如果您有这个选项，请确保您在GPU运行时中运行此笔记本。在Colab中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的Google Cloud项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得$300的免费信用额度，以支付计算/存储费用。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下API：Vertex AI API、Compute Engine API和Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您在本地运行此笔记本，您需要安装[Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter会将以`!`为前缀的行视为shell命令，并会对以`$`为前缀的Python变量进行插值。

#### 设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您也可以更改`REGION`变量，该变量用于笔记本的其余部分操作。以下是支持Vertex AI的区域。我们建议您选择最靠近您的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能无法在使用Vertex AI进行培训时使用多区域存储桶。并非所有区域都支持Vertex AI的所有服务。

了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您在一个实时教程会话中，可能使用一个共享的测试账户或项目。为避免用户在创建的资源中发生名称冲突，您可以为每个实例会话创建一个时间戳，并将时间戳附加到在本教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的Google Cloud帐户

**如果您正在使用Vertex AI Workbench笔记本**，您的环境已经经过身份验证。请跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行身份验证。

**否则**，请按照以下步骤操作：

在Cloud控制台中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐户**。

在**服务帐户名称**字段中输入名称，然后点击**创建**。

在**将此服务帐户授予对项目的访问权限**部分，单击角色下拉列表。在筛选框中输入"Vertex"，并选择**Vertex管理员**。在筛选框中输入"Storage Object Admin"，并选择**存储对象管理员**。

点击创建。下载包含您密钥的JSON文件到本地环境。

在下面的单元格中输入您的服务帐户密钥路径作为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = False
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        IS_COLAB = True
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的是哪种笔记本环境，都需要执行以下步骤。**

当您为 Python 初始化 Vertex AI SDK 时，需要指定一个云存储暂存桶。这个暂存桶是您的数据集和模型资源在会话之间保留的地方。

请在下方设置您的云存储桶的名称。存储桶的名称必须在所有 Google Cloud 项目中是全局唯一的，包括您的组织外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = "gs://" + BUCKET_NAME

只有在您的存储桶不存在时才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查云存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip
import tensorflow as tf

### 初始化Python的Vertex AI SDK

为您的项目和对应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

设置硬件加速器

您可以为训练和预测设置硬件加速器。

将变量`DEPLOY_GPU / DEPLOY_NGPU`设置为使用支持GPU的容器映像以及分配给虚拟机（VM）实例的GPU数量。例如，要使用一个包含4个Nvidia Telsa K80 GPU的GPU容器映像分配给每个VM，您可以指定：

（aip.AcceleratorType.NVIDIA_TESLA_K80，4）

否则，请指定（无，无）以使用一个在CPU上运行的容器映像。

了解有关[您所在区域硬件加速器支持的更多信息](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

*注意*：在TF 2.3之前的GPU支持版本将无法加载此教程中的自定义模型。这是一个已知问题，在TF 2.3中已修复。这是由于生成在serving函数中的静态图操作导致的。如果您在自己的自定义模型上遇到此问题，请使用支持GPU的TF 2.3容器映像。

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预构建的容器

设置用于预测的预构建的Docker容器映像。

要查看最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2.5".replace(".", "-")

if TF[0] == "2":
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置将用于预测的虚拟机的计算资源。
- `机器类型`
     - `n1-standard`：每个 vCPU 3.75GB 的内存。
     - `n1-highmem`： 每个 vCPU 6.5GB 的内存。
     - `n1-highcpu`：每个 vCPU 0.9GB 的内存。
- `vCPUs`：数量为\[2, 4, 8, 16, 32, 64, 96 \]

*注：您还可以使用 n2 和 e2 机器类型进行训练和部署，但它们不支持 GPU。*

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", DEPLOY_COMPUTE)

从公共云存储位置获取预训练模型

为了演示目的，本教程使用了一个预训练的 TensorFlow 1.x 估计器表格分类模型，然后将其上传到`Vertex AI Model`资源。一旦您拥有了一个`Vertex AI Model`资源，该模型可以部署到`Vertex AI Endpoint`资源中。

### 下载预训练模型

从公共云存储中下载预训练的TensorFlow估计器模型工件，然后将模型工件上传到您自己的云存储桶中。

In [ ]:
MODEL_DIR = BUCKET_URI + "/model"

! gsutil cp -r gs://cloud-samples-data/vertex-ai/google-cloud-aiplatform-ci-artifacts/models/penguins/estimator/ {MODEL_DIR}

将TensorFlow估算器模型上传到`Vertex AI Model`资源

最后，您将TFHub模型和服务函数的模型工件上传到`Vertex AI Model`资源。

*注意:*当您将模型工件上传到`Vertex AI Model`资源时，请指定相应的部署容器映像。

In [ ]:
model = aip.Model.upload(
    display_name="example_" + TIMESTAMP,
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
)

print(model)

## 创建`Endpoint`资源

您可以使用`Endpoint.create()`方法创建`Endpoint`资源。最少需要指定端点的显示名称。可选地，您可以指定项目和位置（区域）；否则，设置将继承您使用`init()`方法初始化Vertex AI SDK时设置的值。

在本示例中，指定了以下参数：

- `display_name`：`Endpoint`资源的易读名称。
- `project`：您的项目ID。
- `location`：您的区域。
- `labels`：（可选）以键/值对的形式为`Endpoint`定义的用户定义的元数据。

该方法返回一个 `Endpoint` 对象。

了解更多关于[Vertex AI Endpoints](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)。

In [ ]:
endpoint = aip.Endpoint.create(
    display_name="example_" + TIMESTAMP,
    project=PROJECT_ID,
    location=REGION,
    labels={"your_key": "your_value"},
)

print(endpoint)

## 部署 `Model` 资源到 `Endpoint` 资源。

您可以将一个或多个 `Vertex AI Model` 资源实例部署到同一个端点。每个被部署的 `Vertex AI Model` 资源将拥有自己的用于服务二进制的部署容器。

在下一个示例中，您将 `Vertex AI Model` 资源部署到 `Vertex AI Endpoint` 资源。`Vertex AI Model` 资源已经为其定义了部署容器镜像。为了部署，您需要指定以下额外的配置设置：

- 机器类型。
- （如果有的话）GPU 的类型和数量。
- 静态、手动或自动缩放的 VM 实例。

在此示例中，您将使用指定参数的最小数量来部署模型，具体如下：

- `model`：`Model` 资源。
- `deployed_model_displayed_name`：部署模型实例的可读名称。
- `machine_type`：每个 VM 实例的机器类型。

由于需要预留资源的要求，这可能需要几分钟的时间。

In [ ]:
response = endpoint.deploy(
    model=model,
    deployed_model_display_name="example_" + TIMESTAMP,
    machine_type=DEPLOY_COMPUTE,
)

print(endpoint)

制作预测实例

接下来，您可以使用一个合成示例准备一个预测请求。在这个示例中，模型格式是 TensorFlow 1.x 的估计器格式。该模型格式需要一个请求签名，而`Vertex AI`预构建的 TensorFlow 服务容器不支持这一请求签名。

对于这种模型格式，您可以使用`raw_predict()`方法将一个与模型的服务接口直接匹配的请求传递，具体请求格式如下：

    http_body -> {
        'signature_name' : serving_signature,
        'instances': [ {instance_1}, {instance_2}, ... ]
    }

    instance -> { 'feature_1': value_1, 'feature_2': value_2, ... }

    serving_signature -> "predict"

In [ ]:
import json

from google.api import httpbody_pb2
from google.cloud import aiplatform_v1

DATA = {
    "signature_name": "predict",
    "instances": [
        {
            "island": "DREAM",
            "culmen_length_mm": 36.6,
            "culmen_depth_mm": 18.4,
            "flipper_length_mm": 184.0,
            "body_mass_g": 3475.0,
            "sex": "FEMALE",
        }
    ],
}

http_body = httpbody_pb2.HttpBody(
    data=json.dumps(DATA).encode("utf-8"),
    content_type="application/json",
)

req = aiplatform_v1.RawPredictRequest(
    http_body=http_body, endpoint=endpoint.resource_name
)

### 进行预测

最后，您使用Vertex AI预测服务发出了预测请求。

In [ ]:
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
client_options = {"api_endpoint": API_ENDPOINT}

pred_client = aip.gapic.PredictionServiceClient(client_options=client_options)

response = pred_client.raw_predict(req)
print(response)

## 清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的个别资源。

In [ ]:
delete_bucket = False
delete_model = True
delete_endpoint = True

if delete_endpoint:
    try:
        endpoint.undeploy_all()
        endpoint.delete()
    except Exception as e:
        print(e)

if delete_model:
    try:
        model.delete()
    except Exception as e:
        print(e)

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}